In [0]:
%run /Workspaces/dars_nic_391419_j3w9t_collab/CCU013_02/atlas/00_Load_helpers

### covid cohort from ccu013 

In [0]:
%sql
SELECT
  COUNT(distinct person_id_deid)
FROM
  dars_nic_391419_j3w9t_collab.ccu013_covid_events_demographics_paper_cohort

In [0]:
%sql
select * from dars_nic_391419_j3w9t_collab.ccu013_covid_events_demographics_paper_cohort limit 5;

In [0]:
phe_cohort = spark.sql("""
SELECT 
    person_id_deid,
    date_first as date_first_event,
    first_event as type_first_event,
    date_death,
    death,
    death_covid,
    datediff(date_death, date_first) as ndaysdeath,
    dob,
    age,
    sex,
    ethnic_group,
    IMD_quintile as IMD,
    CASE
        WHEN ethnic_group = 'White'                              THEN 'White'
        WHEN ethnic_group = 'Asian or Asian British'             THEN 'Asian'
        WHEN ethnic_group = 'Black or Black British'             THEN 'Black'
        WHEN (ethnic_group = 'Mixed'  OR ethnic_group = 'Other' OR ethnic_group = 'Chinese') THEN 'Other'
        ELSE 'Unknown'
      END
      AS ethnicity,
    CASE
      WHEN age >= 0  AND age < 18 THEN '0-18'
      WHEN age >= 18 AND age <= 29 THEN '18-29'
      WHEN age >= 30 AND age <= 49 THEN '30-49'
      WHEN age >= 50 AND age <= 69 THEN '50-69'
      ELSE '70plus'
    END age_group,
    CASE
      WHEN date_death IS NOT NULL AND death_covid = 1 THEN 1
      WHEN date_death IS NOT NULL AND death_covid = 0 AND DATEDIFF(date_death, date_first) <= 28 then 2
      ELSE 0
    END outcome
  FROM 
    dars_nic_391419_j3w9t_collab.ccu013_covid_events_demographics_paper_cohort d
""")
display(phe_cohort)

In [0]:
phe_cohort.groupBy('outcome').count().show()

In [0]:
phe_cohort.groupBy('death').count().show()

In [0]:
phe_cohort.createOrReplaceGlobalTempView('ccu013_phe_cohort')

In [0]:
drop_table("ccu013_phe_cohort")

In [0]:
create_table("ccu013_phe_cohort")

In [0]:
%sql
select count(distinct(person_id_deid)) from dars_nic_391419_j3w9t_collab.ccu013_phe_cohort

In [0]:
%sql
select * from dars_nic_391419_j3w9t_collab.ccu013_phe_cohort limit 10